#**Mini-Project 2: Time Series Forecasting using NN, LSTM and CNN** 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Implementation: Data Preprocessing for Fully Connected Neural Network**

######Import required helper functions

In [ ]:
%cd "/content/drive/My Drive/P2_TimeForecast/P2_CodeBase/"
!pip install import-ipynb
import import_ipynb

# importing helper functions for data preprocessing and model visualization
import plotlib_helper as graphplot
import datapreprocess_helper as dataprep

/content/drive/My Drive/P2_TimeForecast/P2_CodeBase
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=27c6f989bf07d682e3d14309e41ef6f83b3fb7d433e0b6d614a7801baebf898d
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
importing Jupyter notebook from plotlib_helper.ipynb
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
importing Jupyter notebook from datapreprocess_helper.ipynb
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###### Define path

In [ ]:
path="/content/drive/My Drive/P2_TimeForecast/P2_CodeBase/data"
dnn_path="/content/drive/My Drive/P2_TimeForecast/P2_CodeBase/dnn"


### **Load data**

In [ ]:
import pandas as pd
import os
import numpy as np 

filename_read = os.path.join(path,"CSC215_P2_Stock_Price.csv")
df = pd.read_csv(filename_read,na_values=['NA','?','-'])

print("dataset:", df.shape)

dataset: (4392, 7)


### **DATA PREPROCESSING STEPS**

#### **1) Drop missing values in dataset set**

#### **2) Drop null values in data set**

#### **3) Drop columns- 'Date' and 'Adj_Close'**

In [ ]:
dropCols=['Date','Adj_Close']
df=dataprep.dataset_clean(df,dropCols)

dataset after dropping NA rows:  (4392, 7)
dataset after dropping  ['Date', 'Adj_Close']  columns:  (4392, 5)


In [ ]:
df

,Open,High,Low,Close,Volume
0,3.812500,4.156250,3.812500,4.125000,3675600
1,4.125000,4.125000,4.000000,4.015625,1077600
2,4.000000,4.031250,3.953125,4.000000,437200
3,4.000000,4.000000,3.843750,3.843750,1883600
4,3.734375,3.734375,3.390625,3.390625,7931600
...,...,...,...,...,...
4387,113.790001,114.099998,112.790001,113.309998,950000
4388,112.519997,113.529999,111.160004,111.870003,1805200
4389,112.029999,112.489998,110.250000,112.230003,2136700
4390,112.459999,112.900002,112.000000,112.339996,1251600


In [ ]:
import copy
x=df
y=copy.deepcopy(df['Close'])


print(x.shape, y.shape)

(4392, 5) (4392,)


In [ ]:
x

,Open,High,Low,Close,Volume
0,3.812500,4.156250,3.812500,4.125000,3675600
1,4.125000,4.125000,4.000000,4.015625,1077600
2,4.000000,4.031250,3.953125,4.000000,437200
3,4.000000,4.000000,3.843750,3.843750,1883600
4,3.734375,3.734375,3.390625,3.390625,7931600
...,...,...,...,...,...
4387,113.790001,114.099998,112.790001,113.309998,950000
4388,112.519997,113.529999,111.160004,111.870003,1805200
4389,112.029999,112.489998,110.250000,112.230003,2136700
4390,112.459999,112.900002,112.000000,112.339996,1251600


In [ ]:
y

0         4.125000
1         4.015625
2         4.000000
3         3.843750
4         3.390625
           ...    
4387    113.309998
4388    111.870003
4389    112.230003
4390    112.339996
4391    113.190002
Name: Close, Length: 4392, dtype: float64

### **ENCODE INPUT COLUMNS**

In [ ]:
#encode input columns
for col in x.columns:
    dataprep.encode_numeric_zscore(x,col)



In [ ]:
x

,Open,High,Low,Close,Volume
0,-0.894209,-0.885022,-0.892306,-0.884364,1.104812
1,-0.883762,-0.886056,-0.885975,-0.888021,-0.497301
2,-0.887941,-0.889159,-0.887558,-0.888543,-0.892217
3,-0.887941,-0.890194,-0.891251,-0.893768,-0.000263
4,-0.896820,-0.898986,-0.906551,-0.908920,3.729366
...,...,...,...,...,...
4387,2.782154,2.754177,2.787407,2.766603,-0.575988
4388,2.739700,2.735310,2.732368,2.718452,-0.048610
4389,2.723320,2.700885,2.701641,2.730490,0.155816
4390,2.737694,2.714457,2.760732,2.734168,-0.390000


In [ ]:
y

0         4.125000
1         4.015625
2         4.000000
3         3.843750
4         3.390625
           ...    
4387    113.309998
4388    111.870003
4389    112.230003
4390    112.339996
4391    113.190002
Name: Close, Length: 4392, dtype: float64

### **FINAL DATA OPERATIONS**

#### **1) Split data rows into batches (7 days) for prediction**

In [ ]:
x = x.to_numpy()
y = y.to_numpy()
print(x.shape, y.shape)
sequence_size=7
x,y=dataprep.to_sequences(sequence_size,x,y)

print(x.shape)

print(type(x))

(4392, 5) (4392,)
(4384, 7, 5)
<class 'numpy.ndarray'>


#### **2) Create 7*5 Input features- Merge 7 days rows as single input record and append Output Column**

In [ ]:
x=x.reshape(x.shape[0],x.shape[1]*x.shape[2])
x.shape


(4384, 35)

In [ ]:
x_df=pd.DataFrame(data=x,index=None)
y_df=pd.DataFrame(data=y,index=None)
result=pd.concat([x_df,y_df], axis=1, ignore_index=True)

result.columns=[*range(0,36,1)]

print(x_df.shape)
print(y_df.shape)
print(result.shape)

(4384, 35)
(4384, 1)
(4384, 36)


In [ ]:
write_filename="P2_preprocessed_NN.csv"
preprocess_write = os.path.join(path,write_filename)
result.to_csv(preprocess_write,index=False)

print("Export Completed- Data exported to:", path, " file name: ", write_filename)


Export Completed- Data exported to: /content/drive/My Drive/P2_TimeForecast/P2_CodeBase/data  file name:  P2_preprocessed_NN.csv
